In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,489 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-06 16:05:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-01-06 16:05:20 (11.2 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [5]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [8]:
useful_data_df = df.filter(df.total_votes >= 20)
useful_data_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-----------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
helpful_df = useful_data_df.filter((useful_data_df.helpful_votes/useful_data_df.total_votes) >= .5)
helpful_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   39823685| RQQ3KVTU5TJ4I|B00005A3L1|     436617004|Bounty Hunter TK4...| Lawn and Garden|          5|           24|         29|   N|                Y|Payed for itself ...|I purchase this m...| 2015-08-31|
|         US|   35875101|R3FELXWV9T5CWE|B00VQVPRH8|     733961147|Multi-Purpose Boo...| Lawn and Garden|          2|    

In [10]:
vine_y = helpful_df.filter(helpful_df.vine == 'Y')
vine_y.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+-------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
vine_n = helpful_df.filter(helpful_df.vine == 'N')
vine_n.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   39823685| RQQ3KVTU5TJ4I|B00005A3L1|     436617004|Bounty Hunter TK4...| Lawn and Garden|          5|           24|         29|   N|                Y|Payed for itself ...|I purchase this m...| 2015-08-31|
|         US|   35875101|R3FELXWV9T5CWE|B00VQVPRH8|     733961147|Multi-Purpose Boo...| Lawn and Garden|          2|    

In [14]:
vine_y.select('star_rating').summary().show()

+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|               386|
|   mean| 4.170984455958549|
| stddev|0.9544360215531862|
|    min|                 1|
|    25%|                 4|
|    50%|                 4|
|    75%|                 5|
|    max|                 5|
+-------+------------------+



In [15]:
vine_n.select('star_rating').summary().show()

+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|             48717|
|   mean|3.6559722478806167|
| stddev| 1.610556758257039|
|    min|                 1|
|    25%|                 2|
|    50%|                 4|
|    75%|                 5|
|    max|                 5|
+-------+------------------+



In [23]:
#total number of helpful reviews
total_reviews = helpful_df.count()
total_reviews

49103

In [25]:
#total number of helpful vine reviews
total_vine = vine_y.count()
total_vine

386

In [27]:
#total number of helpful non-vine reviews
total_no_vine = vine_n.count()
total_no_vine

48717

In [30]:
#vine share of total reviews
(total_vine/total_reviews) * 100

0.7861026821171821

In [29]:
#total helpful five star reviews
total_5star = helpful_df.filter(helpful_df.star_rating == 5).count()
total_5star

24202

In [31]:
#total vine 5* reviews
num_vine_5star = vine_y.filter(vine_y.star_rating == 5).count()
num_vine_5star

176

In [34]:
#total non-vine 5* reviews
num_no_vine_5star = vine_n.filter(vine_n.star_rating == 5).count()
num_no_vine_5star

24026

In [33]:
#vine review share of total five star reviews
(num_vine_5star/total_5star) * 100


0.7272126270556153

In [35]:
#non vine review share of total five star reviews
(num_no_vine_5star/total_5star) * 100


99.27278737294438

In [36]:
#% of vine reviews that are 5*
(num_vine_5star/total_vine) * 100

45.59585492227979

In [37]:
#% of non-vine reviews that are 5*
(num_no_vine_5star/total_no_vine) * 100

49.317486708951705